# Convolution Operation

In [ ]:
import tensorflow as tf

In [ ]:
A = tf.Variable([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
A

<tf.Variable 'Variable:0' shape=(3, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)>

In [ ]:
B = tf.Variable([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
B

<tf.Variable 'Variable:0' shape=(3, 3) dtype=int32, numpy=
array([[ 1,  0, -1],
       [ 1,  0, -1],
       [ 1,  0, -1]], dtype=int32)>

In [ ]:
mult_out = tf.math.multiply(A, B)
mult_out

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 1,  0, -3],
       [ 4,  0, -6],
       [ 7,  0, -9]], dtype=int32)>

In [ ]:
conv_out = tf.math.reduce_sum(mult_out)
conv_out

<tf.Tensor: shape=(), dtype=int32, numpy=-6>

# CNN: MNIST

In [1]:
import tensorflow.keras.datasets.mnist as mnist

In [2]:
(features_train, label_train), (features_test, label_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
label_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [4]:
features_train.shape

(60000, 28, 28)

In [5]:
features_test.shape

(10000, 28, 28)

In [6]:
features_train = features_train.reshape(60000, 28, 28, 1)
features_test = features_test.reshape(10000, 28, 28, 1)

In [7]:
features_train = features_train / 255.0
features_test = features_test / 255.0

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [9]:
np.random.seed(8)
tf.random.set_seed(8)

In [10]:
model = tf.keras.Sequential()

In [11]:
conv_layer1 = layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1))

In [12]:
conv_layer2 = layers.Conv2D(64, (3,3), activation='relu')

In [13]:
fc_layer1 = layers.Dense(128, activation='relu')

In [14]:
fc_layer2 = layers.Dense(10, activation='softmax')

In [15]:
model.add(conv_layer1)
model.add(layers.MaxPooling2D(2, 2))
model.add(conv_layer2)
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Flatten())
model.add(fc_layer1)
model.add(fc_layer2)

In [16]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [17]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [19]:
# Takes about 6.5min to train
model.fit(features_train, label_train, epochs=5, validation_split=0.2, verbose=2)

Epoch 1/5
1500/1500 - 68s - loss: 0.1365 - accuracy: 0.9574 - val_loss: 0.0562 - val_accuracy: 0.9842 - 68s/epoch - 45ms/step
Epoch 2/5
1500/1500 - 61s - loss: 0.0436 - accuracy: 0.9868 - val_loss: 0.0408 - val_accuracy: 0.9876 - 61s/epoch - 40ms/step
Epoch 3/5
1500/1500 - 67s - loss: 0.0295 - accuracy: 0.9907 - val_loss: 0.0333 - val_accuracy: 0.9908 - 67s/epoch - 45ms/step
Epoch 4/5
1500/1500 - 60s - loss: 0.0211 - accuracy: 0.9931 - val_loss: 0.0346 - val_accuracy: 0.9902 - 60s/epoch - 40ms/step
Epoch 5/5
1500/1500 - 60s - loss: 0.0154 - accuracy: 0.9952 - val_loss: 0.0339 - val_accuracy: 0.9912 - 60s/epoch - 40ms/step


In [20]:
model.evaluate(features_test, label_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.0325 - accuracy: 0.9902


[0.03251089155673981, 0.9901999831199646]

# Data Generator: Cats and Dogs

In [42]:
import tensorflow as tf

In [43]:
file_url = 'https://github.com/PacktWorkshops/The-Deep-Learning-Workshop/raw/master/Chapter03/Datasets/Exercise3.03/cats_and_dogs_filtered.zip'

In [44]:
zip_dir = tf.keras.utils.get_file('cats_and_dogs.zip', origin=file_url, extract=True)

68616192/68606236 [==============================] - 0s 0us/step


In [45]:
import pathlib

In [46]:
path = pathlib.Path(zip_dir).parent / 'cats_and_dogs_filtered'
path

PosixPath('/root/.keras/datasets/cats_and_dogs_filtered')

In [47]:
train_dir = path / 'train'
validation_dir = path / 'validation'

In [48]:
train_cats_dir = train_dir / 'cats'
train_dogs_dir = train_dir /'dogs'
validation_cats_dir = validation_dir / 'cats'
validation_dogs_dir = validation_dir / 'dogs'

In [49]:
import os

In [50]:
total_train = len(os.listdir(train_cats_dir)) + len(os.listdir(train_dogs_dir))
total_val = len(os.listdir(validation_cats_dir)) + len(os.listdir(validation_dogs_dir))

In [51]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [52]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [53]:
batch_size=16
img_height = 100
img_width = 100

In [54]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(img_height, img_width),
                                                           class_mode='binary')

Found 2000 images belonging to 2 classes.


In [55]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(img_height, img_width),
                                                              class_mode='binary')

Found 1000 images belonging to 2 classes.


In [56]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [57]:
np.random.seed(8)
tf.random.set_seed(8)

In [58]:
model = tf.keras.Sequential([
    layers.Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width ,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [59]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [60]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [61]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 98, 98, 64)        1792      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 49, 49, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 23, 23, 128)      0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 67712)             0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [62]:
model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/5
125/125 [==============================] - 74s 583ms/step - loss: 0.7217 - accuracy: 0.5490 - val_loss: 0.6576 - val_accuracy: 0.5454
Epoch 2/5
125/125 [==============================] - 72s 572ms/step - loss: 0.6397 - accuracy: 0.6275 - val_loss: 0.6352 - val_accuracy: 0.5917
Epoch 3/5
125/125 [==============================] - 71s 571ms/step - loss: 0.5605 - accuracy: 0.7230 - val_loss: 0.6133 - val_accuracy: 0.6562
Epoch 4/5
125/125 [==============================] - 72s 575ms/step - loss: 0.4603 - accuracy: 0.7915 - val_loss: 0.6973 - val_accuracy: 0.6653
Epoch 5/5
125/125 [==============================] - 72s 575ms/step - loss: 0.3569 - accuracy: 0.8390 - val_loss: 0.6477 - val_accuracy: 0.6905


In [63]:
from tensorflow.keras.datasets import cifar10

In [64]:
(features_train, label_train), (features_test, label_test) = cifar10.load_data()

In [65]:
features_train.shape

(50000, 32, 32, 3)

In [66]:
batch_size=16
img_height = 32
img_width = 32

In [67]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [68]:
train_img_gen = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [69]:
val_img_gen = ImageDataGenerator(rescale=1./255)

In [70]:
train_data_gen = train_img_gen.flow(features_train, label_train, batch_size=batch_size)

In [71]:
val_data_gen = train_img_gen.flow(features_test, label_test, batch_size=batch_size)

In [72]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [73]:
np.random.seed(8)
tf.random.set_seed(8)

In [74]:
model = tf.keras.Sequential([
    layers.Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width ,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [75]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [76]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [77]:
model.fit(
    train_data_gen,
    steps_per_epoch=len(features_train) // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=len(features_test) // batch_size
)

Epoch 1/5
3125/3125 [==============================] - 182s 58ms/step - loss: 1.4822 - accuracy: 0.0969 - val_loss: 1.2190 - val_accuracy: 0.1098
Epoch 2/5
3125/3125 [==============================] - 181s 58ms/step - loss: 1.1735 - accuracy: 0.0964 - val_loss: 1.1134 - val_accuracy: 0.1146
Epoch 3/5
3125/3125 [==============================] - 180s 58ms/step - loss: 1.0756 - accuracy: 0.0970 - val_loss: 1.0597 - val_accuracy: 0.1092
Epoch 4/5
3125/3125 [==============================] - 180s 58ms/step - loss: 1.0111 - accuracy: 0.0974 - val_loss: 1.0145 - val_accuracy: 0.0829
Epoch 5/5
3125/3125 [==============================] - 183s 58ms/step - loss: 0.9646 - accuracy: 0.0998 - val_loss: 1.0256 - val_accuracy: 0.1010


# Image Classification (CIFAR-10) with Data Augmentation

In [27]:
from tensorflow.keras.datasets import cifar10

In [28]:
(features_train, label_train), (features_test, label_test) = cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [29]:
features_train.shape

(50000, 32, 32, 3)

In [30]:
batch_size=16
img_height = 32
img_width = 32

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [32]:
train_img_gen = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [33]:
val_img_gen = ImageDataGenerator(rescale=1./255)

In [34]:
train_data_gen = train_img_gen.flow(features_train, label_train, batch_size=batch_size)

In [35]:
val_data_gen = train_img_gen.flow(features_test, label_test, batch_size=batch_size)

In [36]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [37]:
np.random.seed(8)
tf.random.set_seed(8)

In [38]:
model = tf.keras.Sequential([
    layers.Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width ,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [39]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [40]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [41]:
#model.fit_generator(
model.fit(
    train_data_gen,
    steps_per_epoch=len(features_train) // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=len(features_test) // batch_size
)

Epoch 1/5
3125/3125 [==============================] - 183s 58ms/step - loss: 1.4704 - accuracy: 0.1011 - val_loss: 1.1859 - val_accuracy: 0.0869
Epoch 2/5
3125/3125 [==============================] - 181s 58ms/step - loss: 1.1450 - accuracy: 0.0990 - val_loss: 1.0711 - val_accuracy: 0.1218
Epoch 3/5
3125/3125 [==============================] - 180s 58ms/step - loss: 1.0380 - accuracy: 0.1005 - val_loss: 1.0360 - val_accuracy: 0.0816
Epoch 4/5
3125/3125 [==============================] - 179s 57ms/step - loss: 0.9696 - accuracy: 0.0991 - val_loss: 0.9671 - val_accuracy: 0.1064
Epoch 5/5
3125/3125 [==============================] - 180s 58ms/step - loss: 0.9237 - accuracy: 0.0998 - val_loss: 0.9907 - val_accuracy: 0.1023
